# Cron

Cron is a utility that allows you to schedule commands in a Linux system. You can manage cron jobs using `crontab`, a special utility that handles the file defining jobs for cron.

The following cell runs Alpine Linux, which includes cron by default, providing a playground to experiment with it.

In [1]:
docker run --rm -itd --name crontab_test alpine

2548ef937214e1848ff172baadf7f8b369300dc37cb9a78219530cde84c19ab7


**Note** don't forget to stop the container after all.

In [32]:
docker stop crontab_test

crontab_test


## Cron files

As usually cron configurations are just a specific files in the filesystem. To my mind editing this file is the best option to update cron job.

**Typical location** is `/var/spool/cron/crontabs`, there is separate file for each user.

---

The following cell shows the contents of the `/var/spool/cron/crontabs` folder for the container we're using as an example.

In [14]:
docker exec crontab_test ls -l /var/spool/cron/crontabs

lrwxrwxrwx    1 root     root            21 Dec  5 12:17 /var/spool/cron/crontabs -> ../../../etc/crontabs


There is only one file that describes crontabs for the root user. The next cell lists the contents of this file:

In [4]:
docker exec crontab_test cat /var/spool/cron/crontabs/root

# do daily/weekly/monthly maintenance
# min	hour	day	month	weekday	command
*/15	*	*	*	*	run-parts /etc/periodic/15min
0	*	*	*	*	run-parts /etc/periodic/hourly
0	2	*	*	*	run-parts /etc/periodic/daily
0	3	*	*	6	run-parts /etc/periodic/weekly
0	5	1	*	*	run-parts /etc/periodic/monthly



Typical syntax for linux cron jobs.

## From stdin

You can set your cron schedule using the syntax `crontab -`, which means that `crontab` will take the schedule from standard input.

---

The following cell shows how you can set the crontab through the `echo` command.

In [28]:
docker exec crontab_test sh -c "
echo \"10 * * * * from_echo\" | crontab -
"

Let's check gotten cron schedule.

In [29]:
docker exec crontab_test crontab -l

10 * * * * from_echo


Even better, you can use heredoc syntax to set a multiline cron schedule.

In [30]:
docker exec crontab_test sh -c "
cat <<EOF | crontab -
10 * * * * echo line1
* 10 * * * echo line2
EOF"

Let's check if the schedule from the previous cell was actually added to the cron.

In [31]:
docker exec crontab_test crontab -l

10 * * * * echo line1
* 10 * * * echo line2


## Specify period

To specify the period when each cron job will be executed, you have to use the syntax `*/<amount>` in the corresponding column. For example, `*/2` in the minutes column means that the job will be executed every 2 minutes.  

**Note**: `*/1` is equivalent to just `*`, because `*` means "every."

---

As an example, consider the following Docker container. It runs the `crond` daemon during startup, so the cron jobs will work.

In [5]:
docker run --rm --name run_cron_example -itd alpine \
    crond -fl 2

79d65535b3a13e1ae0f745a14d9f23a1820ca7b06b6fd7ddd731610351eead89


The following cell defines two jobs:

- The first job executes every minute.
- The second job executes every two minutes.

Both jobs print their signatures to the `/test` file, allowing us to check when they have been executed.

In [7]:
docker exec run_cron_example sh -c "
cat << 'EOF' | crontab -
* * * * * echo \"\$(date +%H:%M:%S) every minute\" >> /test
*/2 * * * * echo \"\$(date +%H:%M:%S) every two minutes\" >> /test
EOF"

After a while we can check what we get from our cron jobs:

In [9]:
docker exec run_cron_example cat /test

16:46:00 every minute
16:46:00 every two minutes
16:47:00 every minute
16:48:00 every minute
16:48:00 every two minutes
16:49:00 every minute
16:50:00 every minute
16:50:00 every two minutes


Jobs are executed at the beginning of each minute and every two minutes.

In [10]:
docker stop run_cron_example

run_cron_example
